This uses TF-IDF (Term Frequency-Inverse Document Frequency)

In [8]:
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# --- Configuration ---
# Update these paths if your notebook is in a different folder than your scripts
GLOBAL_CSV_PATH = os.path.join("Datasets", "Holy_Dataset", "anilist_anime_data_complete.csv")
USER_CSV_PATH = "user_animelist_normalized.csv"

print("✅ Libraries imported and paths set.")

✅ Libraries imported and paths set.


Load the data

In [9]:
# 1. Load Data
if not os.path.exists(GLOBAL_CSV_PATH) or not os.path.exists(USER_CSV_PATH):
    print("❌ Missing datasets. Please ensure global and user CSVs exist.")
else:
    df_global = pd.read_csv(GLOBAL_CSV_PATH)
    df_user = pd.read_csv(USER_CSV_PATH)
    print(f"✅ Loaded Global Data: {df_global.shape}")
    print(f"✅ Loaded User Data: {df_user.shape}")

✅ Loaded Global Data: (20099, 62)
✅ Loaded User Data: (328, 12)


Preprocessing & Feature Engineering

In [10]:
# 2. Feature Engineering
if 'idMal' not in df_global.columns:
    print("❌ Global data missing 'idMal' column.")
else:
    # Fill NaN values in text columns to avoid errors
    features = ['genres', 'studios', 'source']
    for feature in features:
        if feature in df_global.columns:
            df_global[feature] = df_global[feature].fillna('')
        else:
            df_global[feature] = ''


    df_global['content_soup'] = (
        df_global['genres'] + " " + 
        df_global['studios'] + " " + 
        df_global['source']
    )
    
    print("✅ Feature engineering complete. 'content_soup' column created.")
    print(f"Sample Entry: {df_global['content_soup'].iloc[0]}")

✅ Feature engineering complete. 'content_soup' column created.
Sample Entry: ["Drama", "Fantasy", "Music", "Romance"] [{"id": 3736, "isMain": false, "node": {"id": 68, "name": "Mushi Production", "isAnimationStudio": true}}] ORIGINAL


Vectorization

In [11]:
# 3. TF-IDF Vectorization
print("... Vectorizing features (TF-IDF)")

tfidf = TfidfVectorizer(stop_words='english')

# Compute the matrix
tfidf_matrix = tfidf.fit_transform(df_global['content_soup'])

print(f"✅ Matrix Created. Shape: {tfidf_matrix.shape}")

... Vectorizing features (TF-IDF)
✅ Matrix Created. Shape: (20099, 41788)


Calculate similarity

In [12]:
# 4. Build User Profile & Calculate Similarity

# Get list of anime the user has completed and liked (Score >= 7)
liked_anime = df_user[(df_user['score'] >= 7) & (df_user['status'] == 'Completed')]
watched_ids = set(df_user['mal_id'].unique())

if liked_anime.empty:
    print("⚠️ You haven't rated enough shows highly (>7) to generate recommendations.")
else:
    print(f"... Analyzing your {len(liked_anime)} favorite shows")

    # Create a Series to map MAL ID to DataFrame Index for fast lookups
    indices = pd.Series(df_global.index, index=df_global['idMal']).drop_duplicates()

    # Store similarity scores: {index: total_similarity_score}
    sim_scores = {}

    for mal_id in liked_anime['mal_id']:
        if mal_id not in indices:
            continue
            
        idx = indices[mal_id]
        
        # Calculate cosine similarity between this liked show and ALL others
        cosine_sim = linear_kernel(tfidf_matrix[idx], tfidf_matrix)
        
        # Get pairwise scores
        scores = list(enumerate(cosine_sim[0]))
        
        # Add to total scores
        for i, score in scores:
            if i in sim_scores:
                sim_scores[i] += score
            else:
                sim_scores[i] = score

    print("✅ Similarity calculations complete.")

... Analyzing your 104 favorite shows
✅ Similarity calculations complete.


Get THE RECCOMENDATİONS

In [13]:
# 5. Sort & Display Results

# Sort by highest similarity score
sorted_scores = sorted(sim_scores.items(), key=lambda x: x[1], reverse=True)

# Get top 10, filtering out what you've already watched
final_recs = []
count = 0

for idx, score in sorted_scores:
    anime_id = df_global.iloc[idx]['idMal']
    
    # Skip if already watched
    if anime_id in watched_ids:
        continue
        
    final_recs.append(df_global.iloc[idx])
    count += 1
    if count >= 10:
        break

# Display Results
print("\n🎉 ML RECOMMENDATIONS FOR YOU 🎉")
print(f"{'Title':<50} | {'Score':<5} | {'Genres'}")
print("-" * 90)

for item in final_recs:
    title = item['title_english'] if pd.notna(item['title_english']) else "Unknown Title"
    # Truncate long titles
    title = (title[:47] + '..') if len(title) > 47 else title
    score = item['averageScore'] if pd.notna(item['averageScore']) else "N/A"
    genres = item['genres'] if pd.notna(item['genres']) else ""
    
    print(f"{title:<50} | {score:<5} | {genres}")


🎉 ML RECOMMENDATIONS FOR YOU 🎉
Title                                              | Score | Genres
------------------------------------------------------------------------------------------
Blue Exorcist: The Movie                           | 73.0  | ["Action", "Fantasy", "Supernatural"]
The Misfit of Demon King Academy: History’s Str..  | 72.0  | ["Action", "Comedy", "Fantasy"]
My Stepmom's Daughter is My Ex                     | 66.0  | ["Comedy", "Romance"]
The Misfit of Demon King Academy Ⅱ: History's S..  | 68.0  | ["Action", "Fantasy"]
TRINITY SEVEN                                      | 67.0  | ["Action", "Comedy", "Ecchi", "Fantasy", "Romance"]
Akira                                              | 79.0  | ["Action", "Adventure", "Horror", "Psychological", "Sci-Fi", "Supernatural"]
Rewrite: Moon and Terra                            | 68.0  | ["Action", "Drama", "Mystery", "Romance", "Supernatural"]
The Misfit of Demon King Academy II (Cour 2)       | 65.0  | ["Action", "Fantasy"

fİNNAly ima be honest
i was looking for something to watch and wrote this and added to the project so yeah